# Image Posterization using K-means Clustering Algorithm

In [ ]:
#import PIL as pil
from PIL import Image
from skimage import io
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering
import joblib

Image compression will be in a method that takes an image and a percentage of pixels to sample. The outputs will be the k selected and the compressed image.

In [ ]:
driving = Image.open("driving.png")

In [ ]:
#How to decide which pixels are selected:
#If the number of pixels to be sampled is 100*k, then divide the number of pixels in the image by that value. Call
#the result n. Every nth pixel will be selected for sampling. To determine if pixel[i,j] is that nth pixel, do
# index = j*width + i to get the pixel index, then mod that value by n. If index % n = 0, you are on that pixel.

#inputs are image and percentage

drivingOriginal = driving
percentage = 0.10  #originally 0.02
pixels = driving.load()

#toSample = 10000
random.seed(a=100) # the random seed should not be based on system time
k = 100   #we would have a loop and iterate k

skip = int(1 / np.sqrt(percentage))
count = 0
size = np.ceil(driving.size[0] / skip) * np.ceil(driving.size[1] / skip)
df_pixels = pd.DataFrame(columns=list('RGB'), index = np.arange(size))

for i in range(0, driving.size[0], skip):
    for j in range(0, driving.size[1], skip):
        i_ = i + random.randint(-3,3)
        if i_>=driving.size[0]:
            i_ = driving.size[0] - 1
        if i_<0:
            i_ = 0
        j_ = j + random.randint(-3,3)
        if j_>=driving.size[1]:
            j_ = driving.size[1] - 1
        if j_<0:
            j_ = 0
            
        df_pixels.iloc[[count]] = [[pixels[i_,j_][0], pixels[i_,j_][1], pixels[i_,j_][2]]]
        #pixels[i_,j_] = (255,255,255)
        count = count + 1
driving

In [ ]:
df_pixels

Now that we have the RGB values of the sampled pixels in the dataset, we run K-means Clustering.
Options: different distance calculations. Euclidean and manhattan and some other one.
Do not force the centers.

In [ ]:
kmeans = KMeans(n_clusters=100) #k
kmeans.fit(df_pixels) #run on df_pixels

kmeans.labels_

In [ ]:
joblib.dump(kmeans, 'kmeans.pkl')
kmeans_loaded = joblib.load('kmeans.pkl')

#predicted = kmeans_loaded.predict([[161,67,19]])

In [ ]:
#print(str(int(kmeans.cluster_centers_[predicted][0][0])) + '    ' + str(kmeans.cluster_centers_[predicted][0])+'     '+str(pixels[0,0]))

In [ ]:
#loop through whole image, predict the group that a pixel belongs in,
#switch the color of the pixel to it's cluster center, record the difference to keep track of variance within groups.
for i in range(0, driving.size[0]):
    for j in range(0, driving.size[1]):
        predicted = kmeans_loaded.predict([[pixels[i,j][0], pixels[i,j][1], pixels[i,j][2]]])
        pixels[i,j] = (round(kmeans.cluster_centers_[predicted][0][0]), round(kmeans.cluster_centers_[predicted][0][1]), round(kmeans.cluster_centers_[predicted][0][2]))

driving

In [ ]:
driving.save("compressed.png", format="png")
print("Hello from the other END")